In [1]:
pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np  


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
# downloaded the images and divide it into training and testing images 
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 27s 1us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 4s 1us/step


In [7]:
# now scaling the images and setting it into 0 and 1 
train_images = train_images / 255.0
test_images = test_images / 255.0

In [8]:
train_images[0].shape  # gives u 28X28 pixel , so we need to reshape it 

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [10]:
def build_model(hp):
    model = keras.Sequential([

        # first adding the convolution layers 
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),  # the values are added for different learning rates 
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    
    return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\91993\AppData\Local\Temp\ipykernel_29876\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [13]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

C:\Users\91993\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# first its a good practise to run the epochs for 3 , then after doing the hyperparameter tuning run for diff. epochs
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)


Trial 5 Complete [00h 23m 49s]
val_accuracy: 0.10316666960716248

Best val_accuracy So Far: 0.903333306312561
Total elapsed time: 00h 51m 01s


In [16]:
model = tuner_search.get_best_models(num_models=1)[0]

C:\Users\91993\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\91993\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96)             │     2,973,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,013,354 (11.50 MB)

 Trainable params: 3,013,354 (11.50 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)  # initial epoch is added to start from 3rd epoch

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 197s 114ms/step - accuracy: 0.9259 - loss: 0.1996 - val_accuracy: 0.9070 - val_loss: 0.2596
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 471s 279ms/step - accuracy: 0.9470 - loss: 0.1418 - val_accuracy: 0.9117 - val_loss: 0.2627
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 944s 546ms/step - accuracy: 0.9626 - loss: 0.1037 - val_accuracy: 0.9107 - val_loss: 0.3295
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 178s 105ms/step - accuracy: 0.9726 - loss: 0.0732 - val_accuracy: 0.9113 - val_loss: 0.3451
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 169s 100ms/step - accuracy: 0.9812 - loss: 0.0519 - val_accuracy: 0.9148 - val_loss: 0.4018
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 143s 85ms/step - accuracy: 0.9870 - loss: 0.0358 - val_accuracy: 0.9113 - val_loss: 0.4536
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 142s 84ms/step - accuracy: 0.9890 - loss: 0.0315 - val_accuracy: 0.9142 - val_loss: 0.4472


In [ ]:
# the above we get the model accuracy as 98.90%